**CAGR Annual_rate of return** :- average annual rate over a period of time 

**Cumulative return** :- total return percentage of gain or loss on an investment 

**Sharpe ratio** :- its a financial metric used to measure the investment risk adjusted return by assessing how much excess return an investment generates per unit of risk 

**Track Drawdown** :- Its a systematically monitor and measure the decline in the value of investment from its highest point to a subsequent lowest point before a new peak is reached 



In [3]:
from IPython.display import display
import pandas as pd
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import norm

# ---------------------------------------------------------
# SAFE DOWNLOAD FUNCTION
# ---------------------------------------------------------
def safe_download(symbols, period="5y"):
    df = yf.download(symbols, period=period, auto_adjust=False)

    if isinstance(symbols, list):
        try:
            return df["Adj Close"].ffill().bfill()
        except:
            return df["Close"].ffill().bfill()

    if "Adj Close" in df.columns:
        return df["Adj Close"].ffill().bfill()
    elif "Close" in df.columns:
        return df["Close"].ffill().bfill()

    raise ValueError("No valid price data found")

# ---------------------------------------------------------
# MONTE CARLO SIMULATION
# ---------------------------------------------------------
def monte_carlo_simulation(portfolio_returns, n_simulations=1000, n_days=252):
    mu = portfolio_returns.mean()
    sigma = portfolio_returns.std()

    simulated_log_returns = np.random.normal(
        mu, sigma, size=(n_days, n_simulations)
    )

    simulated_paths = np.exp(simulated_log_returns.cumsum(axis=0))
    return simulated_paths, simulated_log_returns

# ---------------------------------------------------------
# PORTFOLIO ANALYSIS WITH INVESTMENT AMOUNT
# ---------------------------------------------------------
def portfolio_analysis(
    portfolio_list,
    period="5y",
    risk_free_rate=0.065,
    var_level=0.95,
    invest_amount=1_000_000  # ₹10 lakh default
):

    TRADING_DAYS = 252

    # -----------------------------
    # DATA
    # -----------------------------
    prices = safe_download(portfolio_list, period).dropna()
    log_returns = np.log(prices / prices.shift(1)).dropna()

    portfolio_returns = log_returns.mean(axis=1)
    cumulative_returns = (1 + portfolio_returns).cumprod()

    portfolio_value = invest_amount * cumulative_returns

    # -----------------------------
    # METRICS
    # -----------------------------
    n_days = len(portfolio_returns)

    cagr = (cumulative_returns.iloc[-1] ** (TRADING_DAYS / n_days) - 1) * 100
    vol = portfolio_returns.std() * np.sqrt(TRADING_DAYS) * 100
    sharpe = ((portfolio_returns.mean() * TRADING_DAYS) - risk_free_rate) / (vol / 100)

    drawdown = cumulative_returns / cumulative_returns.cummax() - 1
    max_dd_pct = drawdown.min() * 100
    max_dd_amt = invest_amount * drawdown.min()

    # -----------------------------
    # VISUALIZATIONS
    # -----------------------------
    px.line(portfolio_value, title="Portfolio Value Over Time (₹)").show()

    px.line(
        portfolio_returns.rolling(30).std() * np.sqrt(TRADING_DAYS),
        title="Rolling 30-Day Volatility"
    ).show()

    px.line(drawdown * 100, title="Portfolio Drawdown (%)").show()

    px.histogram(portfolio_returns, nbins=60, title="Return Distribution").show()

    px.imshow(
        log_returns.corr(),
        color_continuous_scale="RdBu",
        title="Asset Correlation Matrix",
        text_auto=True
    ).show()

    # -----------------------------
    # MONTE CARLO
    # -----------------------------
    paths, sim_log_returns = monte_carlo_simulation(portfolio_returns)
    mc_terminal_values = invest_amount * paths[-1]

    fig = go.Figure()
    for i in range(100):
        fig.add_trace(go.Scatter(
            y=invest_amount * paths[:, i],
            mode="lines",
            opacity=0.15,
            showlegend=False
        ))
    fig.update_layout(title="Monte Carlo Simulation – Portfolio Value Paths (₹)")
    fig.show()

    px.histogram(
        mc_terminal_values,
        nbins=50,
        title="Monte Carlo Terminal Portfolio Value Distribution (₹)"
    ).show()

    # -----------------------------
    # VALUE AT RISK (VaR)
    # -----------------------------
    alpha = 1 - var_level

    # % VaR
    hist_var_pct = np.percentile(portfolio_returns, alpha * 100)
    para_var_pct = portfolio_returns.mean() + portfolio_returns.std() * norm.ppf(alpha)
    mc_var_pct = np.percentile(sim_log_returns.flatten(), alpha * 100)

    # ₹ VaR
    hist_var_amt = invest_amount * hist_var_pct
    para_var_amt = invest_amount * para_var_pct
    mc_var_amt = invest_amount * mc_var_pct

    # Expected Shortfall
    hist_es_pct = portfolio_returns[portfolio_returns <= hist_var_pct].mean()
    mc_es_pct = sim_log_returns.flatten()[sim_log_returns.flatten() <= mc_var_pct].mean()

    hist_es_amt = invest_amount * hist_es_pct
    mc_es_amt = invest_amount * mc_es_pct

    # -----------------------------
    # SUMMARY TABLES
    # -----------------------------
    performance_summary = pd.DataFrame({
        "Metric": [
            "CAGR (%)",
            "Annual Volatility (%)",
            "Sharpe Ratio",
            "Max Drawdown (%)",
            "Max Drawdown (₹)"
        ],
        "Value": [
            round(cagr,4),
            round(vol,4),
            round(sharpe,4),
            round(max_dd_pct,4),
            round(max_dd_amt,2)
        ]
    })

    var_summary = pd.DataFrame({
        "Risk Measure": [
            "Historical VaR (₹)",
            "Parametric VaR (₹)",
            "Monte Carlo VaR (₹)",
            "Historical ES (₹)",
            "Monte Carlo ES (₹)"
        ],
        "Loss Amount": [
            round(hist_var_amt,2),
            round(para_var_amt,2),
            round(mc_var_amt,2),
            round(hist_es_amt,2),
            round(mc_es_amt,2)
        ]
    })

    display(performance_summary)
    display(var_summary)

    return performance_summary, var_summary

# ---------------------------------------------------------
# RUN
# ---------------------------------------------------------
portfolio_list = [
    "BDL.NS","CANBK.NS","BEL.NS","ADANIPOWER.NS","ITC.NS","ASHOKLEY.NS",
    "LTF.NS","BHEL.NS","BIOCON.NS","SAIL.NS","BANKINDIA.NS",
    "IREDA.NS","INDUSTOWER.NS","SWIGGY.NS","NYKAA.NS",
    "FEDERALBNK.NS","AARTIIND.NS"
]

portfolio_analysis(
    portfolio_list=portfolio_list,
    period="5y",
    risk_free_rate=0.0635,
    var_level=0.95,
    invest_amount=1_000_000  # ₹10 lakh
)


[*********************100%***********************]  17 of 17 completed


,Metric,Value
0,CAGR (%),21.8757
1,Annual Volatility (%),20.6611
2,Sharpe Ratio,0.7549
3,Max Drawdown (%),-29.2450
4,Max Drawdown (₹),-292449.5700


,Risk Measure,Loss Amount
0,Historical VaR (₹),-21770.77
1,Parametric VaR (₹),-20537.35
2,Monte Carlo VaR (₹),-20563.67
3,Historical ES (₹),-31799.03
4,Monte Carlo ES (₹),-26004.44


(                  Metric        Value
 0               CAGR (%)      21.8757
 1  Annual Volatility (%)      20.6611
 2           Sharpe Ratio       0.7549
 3       Max Drawdown (%)     -29.2450
 4       Max Drawdown (₹) -292449.5700,
           Risk Measure  Loss Amount
 0   Historical VaR (₹)    -21770.77
 1   Parametric VaR (₹)    -20537.35
 2  Monte Carlo VaR (₹)    -20563.67
 3    Historical ES (₹)    -31799.03
 4   Monte Carlo ES (₹)    -26004.44)

In [4]:
#portfolio optimization 

more_stocks = ["IDFCFIRSTB.NS", "PNB.NS", "AUBANK.NS", "HDFCBANK.NS", "ICICIBANK.NS", "BAJAJFINSV.NS", "CIPLA.NS", "DRREDDY.NS" ,"IPCALAB.NS", "LUPIN.NS", "ZYDUSLIFE.NS", "SBICARD.NS", "ASIANPAINT.NS", "HINDUNILVR.NS", "TCS.NS", "INFY.NS", "LT.NS", "MARUTI.NS", "SUNPHARMA.NS", "WIPRO.NS", "ADANIGREEN.NS", "ADANIPOWER.NS",     "DLF.NS", "DMART.NS", "DRREDDY.NS", "EIDPARRY.NS",
    "EICHERMOT.NS", "EIHOTEL.NS", "ELGIEQUIP.NS", "EMAMILTD.NS", "FEDERALBNK.NS",
    "GAIL.NS", "GLAND.NS", "GLAXO.NS", "GLENMARK.NS", "GNFC.NS",
    "GODREJAGRO.NS", "GODREJCP.NS", "GODREJPROP.NS", "GRANULES.NS", "GRASIM.NS",
    "HAL.NS", "HAVELLS.NS", "HCLTECH.NS", "HDFCAMC.NS", "HDFCBANK.NS",
    "HEROMOTOCO.NS", "HINDALCO.NS", "HINDUNILVR.NS", "ICICIGI.NS", "ICICIBANK.NS",
    "IEX.NS", "INDHOTEL.NS", "INDIGO.NS", "INDOWIND.NS", "INDUSTOWER.NS",
    "INDUSINDBK.NS", "INFY.NS", "INOXGREEN.NS", "INOXWIND.NS",
    "IOC.NS", "IPCALAB.NS", "IRCTC.NS", "IRFC.NS", "ITC.NS",
    "JINDALSTEL.NS", "JSWENERGY.NS", "JSWSTEEL.NS", "KOTAKBANK.NS", "KPIGREEN.NS",
    "LAURUSLABS.NS", "LTIM.NS", "LT.NS", "LUPIN.NS", "M&M.NS",
    "MANKIND.NS", "MARICO.NS", "MARUTI.NS", "MAXHEALTH.NS",
    "MOREPENLAB.NS", "MOTHERSON.NS", "MPHASIS.NS", "NATCOPHARM.NS", "NAUKRI.NS",
    "NESTLEIND.NS", "NHPC.NS", "NMDC.NS", "NTPC.NS",
    "OBEROIRLTY.NS", "OLECTRA.NS", "ONGC.NS", "ORCHPHARMA.NS",
    "PAGEIND.NS", "PAYTM.NS", "PFC.NS", "PFIZER.NS", "PNB.NS",
    "PNBHOUSING.NS", "POLICYBZR.NS", "POWERGRID.NS", "RECLTD.NS", "RELAXO.NS",
    "RELIANCE.NS", "SANOFI.NS", "SBIN.NS", "SBILIFE.NS",
    "SHREECEM.NS", "SIEMENS.NS", "SUNPHARMA.NS",
    "SUZLON.NS", "SWSOLAR.NS", "TATACONSUM.NS", "TATAPOWER.NS", "TATASTEEL.NS",
    "TCS.NS", "TECHM.NS", "TITAN.NS", "TORNTPHARM.NS", "TORNTPOWER.NS",
    "TRENT.NS",  "UPL.NS", "VBL.NS", "VEDL.NS",
    "VGUARD.NS", "WESTLIFE.NS",
    "WHIRLPOOL.NS", "WIPRO.NS", "YESBANK.NS", "ZYDUSLIFE.NS"]
portfolio_list.extend(more_stocks)

df_port = yf.download(portfolio_list, period="6mo", auto_adjust=False)["Adj Close"]


[*********************100%***********************]  128 of 128 completed


In [5]:
# stock selection and optimization

benchmark_sharpe =  0.99
optimized_stocks = []

# Volatility for each stock 
log_returns = np.log(df_port / df_port.shift(1)).dropna()
daily_volatility = log_returns.std()
annual_volatility_per = (daily_volatility * np.sqrt(df_port.shape[0])) * 100 
annual_volatility= (daily_volatility * np.sqrt(df_port.shape[0]))
# Cumulative return (decimal per stock)
cumulative_return = (df_port.iloc[-1] - df_port.iloc[0]) / df_port.iloc[0]
percentage_cumulative_return = cumulative_return * 100

#sharpe ratio
risk_free_rate = 0.0653
portfolio_return_mean_ = (log_returns.mean() * df_port.shape[0]) - risk_free_rate
sharpe_ratio = portfolio_return_mean_ / annual_volatility


# Combined filter using & operator
filtered = sharpe_ratio < benchmark_sharpe

# Extract ticker names
optimized_stocks = list(filtered[filtered].index)

# Remove these from portfolio
portfolio_list = [stock for stock in portfolio_list if stock not in optimized_stocks]

# Download updated portfolio data
df_port = yf.download(portfolio_list, period="6mo", auto_adjust=False)["Adj Close"]

print("Removed:", optimized_stocks)
print("Updated & Downloaded portfolio:", portfolio_list)


[*********************100%***********************]  15 of 15 completed

Removed: ['AARTIIND.NS', 'ADANIGREEN.NS', 'ADANIPOWER.NS', 'ASIANPAINT.NS', 'AUBANK.NS', 'BAJAJFINSV.NS', 'BDL.NS', 'BEL.NS', 'BHEL.NS', 'BIOCON.NS', 'CIPLA.NS', 'DLF.NS', 'DMART.NS', 'DRREDDY.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'GAIL.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GNFC.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJPROP.NS', 'GRANULES.NS', 'GRASIM.NS', 'HAL.NS', 'HAVELLS.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'INDHOTEL.NS', 'INDIGO.NS', 'INDOWIND.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXWIND.NS', 'IOC.NS', 'IPCALAB.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'JINDALSTEL.NS', 'JSWENERGY.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'KPIGREEN.NS', 'LT.NS', 'LTIM.NS', 'LUPIN.NS', 'M&M.NS', 'MANKIND.NS', 'MARICO.NS', 'MAXHEALTH.NS', 'MOREPENLAB.NS', 'MOTHERSON.NS', 'MPHASIS.NS', 'NATCOPHARM.NS', 'NAUKRI.NS', 'NESTLEIND.NS', 'NHPC.NS', 'NTPC.NS', 'NYKAA.NS', 

In [6]:
# as per 16 November 2025 the risk free rate in india is 6.53%
portfolio_analysis(
    portfolio_list=portfolio_list,
    period="1y",
    risk_free_rate=0.0653,
    var_level=0.95,
    invest_amount=1_000_000  # ₹10 lakh
)

[*********************100%***********************]  15 of 15 completed


,Metric,Value
0,CAGR (%),45.6393
1,Annual Volatility (%),18.8957
2,Sharpe Ratio,1.7398
3,Max Drawdown (%),-12.8044
4,Max Drawdown (₹),-128043.6500


,Risk Measure,Loss Amount
0,Historical VaR (₹),-20150.93
1,Parametric VaR (₹),-18015.32
2,Monte Carlo VaR (₹),-18057.08
3,Historical ES (₹),-28131.16
4,Monte Carlo ES (₹),-23005.97


(                  Metric        Value
 0               CAGR (%)      45.6393
 1  Annual Volatility (%)      18.8957
 2           Sharpe Ratio       1.7398
 3       Max Drawdown (%)     -12.8044
 4       Max Drawdown (₹) -128043.6500,
           Risk Measure  Loss Amount
 0   Historical VaR (₹)    -20150.93
 1   Parametric VaR (₹)    -18015.32
 2  Monte Carlo VaR (₹)    -18057.08
 3    Historical ES (₹)    -28131.16
 4   Monte Carlo ES (₹)    -23005.97)

In [10]:
# Markowite portfolio model

from scipy.optimize import minimize

tickers = df_port.columns.tolist()
num_assets = len(tickers)

returns = df_port.pct_change().dropna()
mean_returns = returns.mean() * len(df_port)        # Annualized expected return
std_returns = returns.std() * np.sqrt(len(df_port)) # Annualized risk (std dev)
cov_matrix = returns.cov() * len(df_port)           # Annualized covariance matrix

# -----------------------------
#  Portfolio functions
# -----------------------------
def portfolio_return(weights):
    return np.sum(weights * mean_returns)

def portfolio_variance(weights):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

bounds = tuple((0, 1) for _ in range(num_assets))

target_returns = np.linspace(mean_returns.min(), mean_returns.max(), 100)
risks = []
weights_record = []

for r in target_returns:
    constraints = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'eq', 'fun': lambda x: portfolio_return(x) - r}
    )
    result = minimize(portfolio_variance, num_assets * [1. / num_assets],
                      method='SLSQP', bounds=bounds, constraints=constraints)
    if result.success:
        risks.append(np.sqrt(result.fun))
        weights_record.append(result.x)

# -----------------------------
#  Convert to percentage form
# -----------------------------
risks_percent = np.array(risks) * 100
returns_percent = np.array(target_returns) * 100

# -----------------------------
#  DataFrame for individual stocks
# -----------------------------
stock_df = pd.DataFrame({
    'Ticker': [ticker.replace('.NS', '') for ticker in tickers],
    'Expected Return (%)': (mean_returns * 100).round(2),
    'Risk (Volatility %)': (std_returns * 100).round(2)
})

# -----------------------------
# 5️⃣ Plotly Visualization
# -----------------------------
fig = go.Figure()

# Efficient Frontier
fig.add_trace(go.Scatter(
    x=risks_percent, y=returns_percent,
    mode='lines',
    name='Efficient Frontier',
    line=dict(color='crimson', width=4, dash='solid'),
    hovertemplate='<b>Risk:</b> %{x:.2f}%<br><b>Return:</b> %{y:.2f}%<extra></extra>'
))

# Individual Stocks
fig.add_trace(go.Scatter(
    x=std_returns * 100, y=mean_returns * 100,
    mode='markers+text',
    textposition='top center',
    marker=dict(size=11, color='royalblue', line=dict(width=1, color='white')),
    name='Individual Stocks',
    hovertemplate='<b>%{text}</b><br>Risk: %{x:.2f}%<br>Return: %{y:.2f}%<extra></extra>'
))

# -----------------------------
#  Layout Aesthetic Improvements
# -----------------------------
fig.update_layout(
    title=dict(text='<b> Pre optimization Efficient Frontier with Individual Stocks</b>', x=0.5, font=dict(size=22)),
    xaxis=dict(title='Risk (Standard Deviation %)', tickformat='.1f'),
    yaxis=dict(title='Expected Return (%)', tickformat='.1f'),
    template='plotly_white',
    plot_bgcolor='rgba(240,240,240,0.95)',
    paper_bgcolor='rgba(255,255,255,1)',
    font=dict(family='Verdana', size=13),
    hoverlabel=dict(bgcolor='white', font_size=13)
)

fig.show()

# -----------------------------
#  Styled DataFrame Display
# -----------------------------
stock_df.style.background_gradient(cmap='RdYlGn', subset=['Expected Return (%)']) \
         .background_gradient(cmap='Blues', subset=['Risk (Volatility %)'])


,Ticker,Expected Return (%),Risk (Volatility %)
Ticker,,,
ASHOKLEY.NS,ASHOKLEY,43.530000,19.840000
BANKINDIA.NS,BANKINDIA,24.230000,17.190000
CANBK.NS,CANBK,31.990000,16.940000
EICHERMOT.NS,EICHERMOT,27.440000,12.760000
FEDERALBNK.NS,FEDERALBNK,22.370000,13.640000
HEROMOTOCO.NS,HEROMOTOCO,36.920000,17.740000
HINDALCO.NS,HINDALCO,29.290000,14.980000
INOXGREEN.NS,INOXGREEN,34.090000,27.000000
LAURUSLABS.NS,LAURUSLABS,40.780000,17.950000
